In [ ]:
#!pip install numpy implicit tqdm

In [ ]:
#!pip install -U scikit-learn

In [ ]:
#!pip install scikit-learn=1.0.2

In [ ]:
!pip install -U torch

In [ ]:
#!pip install polars==0.19.9 
# !pip install -U polars

In [ ]:
#!pip install ray

In [ ]:
#!pip install recbole

In [ ]:
#!pip install ipywidgets

In [ ]:
#!pip install cloudpickle

In [ ]:
#%pip install MetaUtils

In [1]:
import pickle

In [2]:
with open('list_good_users.pkl', 'rb') as f:
    list_good_users = pickle.load(f)

list_good_users

array(['u_0', 'u_100', 'u_10000', ..., 'u_999994', 'u_999996', 'u_999999'],
      dtype=object)

In [3]:
len(list_good_users)

819386

In [2]:
import polars as pl
import numpy as np
from tqdm import tqdm
#from tqdm.notebook import tqdm   # лучше вывод?
from scipy.sparse import csr_matrix
import implicit

RANDOM_STATE = 42
N_PREDICTIONS = 100
DATASET_NAME = "hh_recsys"

In [5]:
pl.show_versions()

--------Version info---------
Polars:               0.20.7
Index type:           UInt32
Platform:             Linux-5.10.0-21-amd64-x86_64-with-glibc2.31
Python:               3.9.2 (default, Feb 28 2021, 17:03:44) 
[GCC 10.2.1 20210110]

----Optional dependencies----
adbc_driver_manager:  <not installed>
cloudpickle:          <not installed>
connectorx:           <not installed>
deltalake:            <not installed>
fsspec:               2023.6.0
gevent:               <not installed>
hvplot:               <not installed>
matplotlib:           3.7.1
numpy:                1.23.5
openpyxl:             <not installed>
pandas:               1.5.3
pyarrow:              13.0.0
pydantic:             1.10.9
pyiceberg:            <not installed>
pyxlsb:               <not installed>
sqlalchemy:           2.0.20
xlsx2csv:             <not installed>
xlsxwriter:           <not installed>


In [6]:
print(pl.__version__ == "0.19.9")

False


In [3]:
import datetime
import os

import torch
from recbole.data.interaction import Interaction
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.quick_start import run_recbole
from recbole.quick_start.quick_start import load_data_and_model  
from recbole.utils.case_study import full_sort_topk
from recbole.utils.utils import get_trainer

2024-02-10 03:03:14.602620: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-10 03:03:14.602673: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-10 03:03:14.603953: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-10 03:03:14.611260: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-10 03:03:15.549358: W tensorflow/comp

In [10]:
train = pl.read_parquet("./hh_recsys_train_hh.pq", low_memory=True)
test = pl.read_parquet("./hh_recsys_test_hh.pq", low_memory=True)
vacancies = pl.read_parquet("./hh_recsys_vacancies.pq", low_memory=True)

In [4]:
train

user_id,session_id,vacancy_id,action_type,action_dt
str,str,list[str],list[i64],list[datetime[ns]]
"""u_332060""","""s_28301374""","[""v_2571684"", ""v_488179"", … ""v_2633899""]","[2, 2, … 2]","[2023-11-01 00:40:58.105, 2023-11-01 00:58:13.091, … 2023-11-01 01:35:54.456]"
"""u_1057881""","""s_33868982""","[""v_665861""]",[2],[2023-11-01 00:23:51.452]
"""u_1036784""","""s_32474802""","[""v_2594840""]",[2],[2023-11-01 00:52:34.023]
"""u_786220""","""s_14060785""","[""v_1473781"", ""v_1622905"", … ""v_2394151""]","[1, 2, … 2]","[2023-11-01 00:58:20.793, 2023-11-01 01:17:04.774, … 2023-11-01 01:17:40.149]"
"""u_639152""","""s_23205986""","[""v_695738"", ""v_22433"", … ""v_1078110""]","[2, 1, … 2]","[2023-11-01 01:14:20.828, 2023-11-01 00:54:08.636, … 2023-11-01 01:00:53.660]"
"""u_456512""","""s_6053713""","[""v_2267837"", ""v_1724756"", ""v_990702""]","[2, 2, 2]","[2023-11-01 01:37:09.157, 2023-11-01 01:15:55.336, 2023-11-01 01:40:32.097]"
"""u_584261""","""s_11620144""","[""v_902380"", ""v_1488750"", … ""v_840395""]","[2, 2, … 2]","[2023-11-01 01:09:49.508, 2023-11-01 01:25:00.774, … 2023-11-01 01:12:01.125]"
"""u_733524""","""s_13468241""","[""v_1460424"", ""v_996518"", … ""v_2273974""]","[2, 2, … 2]","[2023-11-01 02:12:43.970, 2023-11-01 02:50:33.283, … 2023-11-01 02:50:34.962]"
"""u_1114237""","""s_23351853""","[""v_695101"", ""v_154777""]","[2, 2]","[2023-11-01 04:18:48.790, 2023-11-01 04:17:57.514]"


In [11]:
train = train.filter(pl.col("vacancy_id").list.len()<52)

In [28]:
# train = train.filter(pl.col("user_id").is_in(list_good_users.tolist()))

In [5]:
train.select(pl.col("vacancy_id").list.len().mean())

vacancy_id
f64
5.142407


In [6]:
train

user_id,session_id,vacancy_id,action_type,action_dt
str,str,list[str],list[i64],list[datetime[ns]]
"""u_1057881""","""s_33868982""","[""v_665861""]",[2],[2023-11-01 00:23:51.452]
"""u_1036784""","""s_32474802""","[""v_2594840""]",[2],[2023-11-01 00:52:34.023]
"""u_786220""","""s_14060785""","[""v_1473781"", ""v_1622905"", … ""v_2394151""]","[1, 2, … 2]","[2023-11-01 00:58:20.793, 2023-11-01 01:17:04.774, … 2023-11-01 01:17:40.149]"
"""u_639152""","""s_23205986""","[""v_695738"", ""v_22433"", … ""v_1078110""]","[2, 1, … 2]","[2023-11-01 01:14:20.828, 2023-11-01 00:54:08.636, … 2023-11-01 01:00:53.660]"
"""u_456512""","""s_6053713""","[""v_2267837"", ""v_1724756"", ""v_990702""]","[2, 2, 2]","[2023-11-01 01:37:09.157, 2023-11-01 01:15:55.336, 2023-11-01 01:40:32.097]"
"""u_584261""","""s_11620144""","[""v_902380"", ""v_1488750"", … ""v_840395""]","[2, 2, … 2]","[2023-11-01 01:09:49.508, 2023-11-01 01:25:00.774, … 2023-11-01 01:12:01.125]"
"""u_733524""","""s_13468241""","[""v_1460424"", ""v_996518"", … ""v_2273974""]","[2, 2, … 2]","[2023-11-01 02:12:43.970, 2023-11-01 02:50:33.283, … 2023-11-01 02:50:34.962]"
"""u_1114237""","""s_23351853""","[""v_695101"", ""v_154777""]","[2, 2]","[2023-11-01 04:18:48.790, 2023-11-01 04:17:57.514]"
"""u_407319""","""s_23566251""","[""v_268677"", ""v_692096"", … ""v_416576""]","[2, 2, … 2]","[2023-11-01 04:56:41.605, 2023-11-01 04:57:49.326, … 2023-11-01 05:15:18.660]"


In [36]:
test

user_id,session_id,vacancy_id,action_type,action_dt,predictions
str,str,list[str],list[i64],list[datetime[ns]],list[str]
"""u_482520""","""s_25018731""","[""v_2597196"", ""v_1223061"", ""v_1223061""]","[2, 2, 1]","[2023-11-19 12:03:13.089, 2023-11-19 12:03:30.396, 2023-11-19 12:05:03.473]","[""v_507544"", ""v_2483238"", … ""v_394838""]"
"""u_582132""","""s_481216""","[""v_470400"", ""v_470400"", ""v_1530783""]","[2, 1, 2]","[2023-11-21 15:39:47.981, 2023-11-21 15:43:57.620, 2023-11-21 15:44:37.982]","[""v_1748350"", ""v_1693409"", … ""v_963346""]"
"""u_212584""","""s_16918781""","[""v_1572055"", ""v_1572055"", … ""v_1329494""]","[2, 3, … 1]","[2023-11-16 08:41:47.031, 2023-11-16 08:45:21.403, … 2023-11-16 09:05:27.379]","[""v_2075726"", ""v_665187"", … ""v_1960845""]"
"""u_425177""","""s_17505104""","[""v_1375331"", ""v_1922852""]","[2, 2]","[2023-11-17 12:42:18.513, 2023-11-17 12:43:52.744]","[""v_1889365"", ""v_638883"", … ""v_2715616""]"
"""u_700997""","""s_15528830""","[""v_2152997"", ""v_2152997"", ""v_1217630""]","[2, 1, 2]","[2023-11-16 17:22:53.530, 2023-11-16 17:23:19.885, 2023-11-16 17:23:33.472]","[""v_2125504"", ""v_1967086"", … ""v_1677861""]"
"""u_687201""","""s_11187326""","[""v_2602959"", ""v_549337"", … ""v_212060""]","[2, 2, … 1]","[2023-11-21 14:41:30.546, 2023-11-21 15:00:25.506, … 2023-11-21 15:24:56.271]","[""v_1503324"", ""v_915223"", … ""v_2503802""]"
"""u_573219""","""s_27394215""","[""v_1836184"", ""v_2733253"", … ""v_2297339""]","[1, 2, … 2]","[2023-11-21 18:48:43.424, 2023-11-21 18:49:12.552, … 2023-11-21 19:48:55.704]","[""v_2234498"", ""v_2171895"", … ""v_1016980""]"
"""u_1082782""","""s_30341164""","[""v_1026036"", ""v_1026036"", … ""v_206700""]","[2, 1, … 2]","[2023-11-17 02:54:19.561, 2023-11-17 02:54:50.486, … 2023-11-17 03:21:22.008]","[""v_1949840"", ""v_1492438"", … ""v_364150""]"
"""u_81444""","""s_1909964""","[""v_2497384"", ""v_1480870"", … ""v_1500697""]","[2, 2, … 1]","[2023-11-15 12:38:34.579, 2023-11-15 13:03:41.360, … 2023-11-15 13:05:15.916]","[""v_1949840"", ""v_987212"", … ""v_900375""]"


In [12]:
train = train.vstack(test)

In [37]:
train

user_id,session_id,vacancy_id,action_type,action_dt
str,str,list[str],list[i64],list[datetime[ns]]
"""u_1057881""","""s_33868982""","[""v_665861""]",[2],[2023-11-01 00:23:51.452]
"""u_1036784""","""s_32474802""","[""v_2594840""]",[2],[2023-11-01 00:52:34.023]
"""u_786220""","""s_14060785""","[""v_1473781"", ""v_1622905"", … ""v_2394151""]","[1, 2, … 2]","[2023-11-01 00:58:20.793, 2023-11-01 01:17:04.774, … 2023-11-01 01:17:40.149]"
"""u_639152""","""s_23205986""","[""v_695738"", ""v_22433"", … ""v_1078110""]","[2, 1, … 2]","[2023-11-01 01:14:20.828, 2023-11-01 00:54:08.636, … 2023-11-01 01:00:53.660]"
"""u_456512""","""s_6053713""","[""v_2267837"", ""v_1724756"", ""v_990702""]","[2, 2, 2]","[2023-11-01 01:37:09.157, 2023-11-01 01:15:55.336, 2023-11-01 01:40:32.097]"
"""u_584261""","""s_11620144""","[""v_902380"", ""v_1488750"", … ""v_840395""]","[2, 2, … 2]","[2023-11-01 01:09:49.508, 2023-11-01 01:25:00.774, … 2023-11-01 01:12:01.125]"
"""u_733524""","""s_13468241""","[""v_1460424"", ""v_996518"", … ""v_2273974""]","[2, 2, … 2]","[2023-11-01 02:12:43.970, 2023-11-01 02:50:33.283, … 2023-11-01 02:50:34.962]"
"""u_1114237""","""s_23351853""","[""v_695101"", ""v_154777""]","[2, 2]","[2023-11-01 04:18:48.790, 2023-11-01 04:17:57.514]"
"""u_407319""","""s_23566251""","[""v_268677"", ""v_692096"", … ""v_416576""]","[2, 2, … 2]","[2023-11-01 04:56:41.605, 2023-11-01 04:57:49.326, … 2023-11-01 05:15:18.660]"


In [14]:
vacancies

vacancy_id,name,company.id,description,keySkills.keySkill,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience
str,str,str,str,list[str],i64,i64,str,str,str,str,str,str
"""v_862116""","""Смотритель муз…","""c_162972""","""<strong>Обязан…","[""Пользователь ПК"", ""Работа в команде"", … ""PR-консультирование""]",16500,null,"""RUR""","""a_4761""","""ar_33""","""full""","""fullDay""","""noExperience"""
"""v_288642""","""Ведущий менедж…","""c_208672""","""<p><strong>Воз…","[""Активные продажи"", ""Холодные продажи"", … ""Организация мероприятий""]",50000,null,"""RUR""","""a_744""","""ar_2""","""full""","""fullDay""","""noExperience"""
"""v_1840054""","""Бухгалтер (по …","""c_198109""","""<strong>Обязан…",null,50000,65000,"""RUR""","""a_6223""","""ar_78""","""full""","""fullDay""","""between3And6"""
"""v_2346232""","""Пекарь (Токсов…","""c_6137""","""<p><strong>Для…",null,38500,42000,"""RUR""","""a_4795""","""ar_51""","""full""","""fullDay""","""noExperience"""
"""v_312507""","""Торговый предс…","""c_206699""","""<p>Компания ТД…","[""Продуктивность"", ""Клиентоориентированность"", … ""Развитие продаж""]",60000,null,"""RUR""","""a_6837""","""ar_4""","""full""","""fullDay""","""between1And3"""
"""v_2323706""","""Менеджер по ка…","""c_244141""","""<p><strong>X</…","[""Контроль качества"", ""Производственный контроль"", … ""Лабораторная диагностика""]",88000,null,"""RUR""","""a_5920""","""ar_64""","""full""","""fullDay""","""between1And3"""
"""v_704266""","""Менеджер по пр…","""c_192038""","""<p>Если ты ище…","[""Активные продажи"", ""Поиск и привлечение клиентов"", … ""Проведение переговоров""]",30000,100000,"""RUR""","""a_5398""","""ar_71""","""full""","""fullDay""","""between1And3"""
"""v_525733""","""Врач по ЛФК""","""c_5101""","""<strong>Обязан…",null,null,null,null,"""a_5043""","""ar_96""","""full""","""fullDay""","""between3And6"""
"""v_2091086""","""Монтажник СТиЖ…","""c_230890""","""<em><strong>Ст…",null,100000,null,"""RUR""","""a_500""","""ar_36""","""full""","""flyInFlyOut""","""between1And3"""


In [15]:
vacancies = vacancies.with_columns(pl.col("compensation.from").fill_null(strategy="min"))
vacancies = vacancies.with_columns(pl.when(pl.col("compensation.to").is_null()).then(pl.col("compensation.from")).otherwise(pl.col("compensation.to")).alias("compensation.to"))
vacancies = vacancies.with_columns(pl.col("compensation.currencyCode").fill_null("RUR"))
# не можем это все сделать в одну строку потому что есть строки где и from и to ==  null - тогда они обработаются некорректно

keySkills.keySkill и area.regionId могут быть null, остальные вроде бы нет.
(Проверим:)

In [16]:
vacancies.filter(pl.col("name").is_null())

vacancy_id,name,company.id,description,keySkills.keySkill,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience
str,str,str,str,list[str],i64,i64,str,str,str,str,str,str


In [17]:
vacancies = vacancies.with_columns(pl.col("description").str.replace_all("<.*?>", "").  # убираем html-тэги
                                   str.replace_all("&[A-Za-z0-9#]+;", "").   # убираем entity вроде  &quot;
                                   str.replace_all("[[:punct:]]", "").   # убираем знаки препинания
                                   str.to_lowercase())                       # лучше все к нижнему регистру
#                                  str.replace_all("&quot;", "")) # - можно было бы и так, последовательно, но их бывает много разных

In [18]:
vacancies = vacancies.with_columns(pl.col("keySkills.keySkill").list.join(' ').str.to_lowercase())  # ключевые навыки записываем в строку через пробел и в нижнем регистре
# если .join(', ') - то будет через запятую

In [19]:
with pl.Config(fmt_str_lengths=50):
#    print(vacancies['description'])
    print(vacancies['keySkills.keySkill'])

shape: (2_734_129,)
Series: 'keySkills.keySkill' [str]
[
	"пользователь пк работа в команде умение планирова…
	"активные продажи холодные продажи кредитные догов…
	null
	null
	"продуктивность клиентоориентированность управлени…
	"контроль качества производственный контроль навык…
	"активные продажи поиск и привлечение клиентов под…
	null
	null
	"умение работать в команде уборка строительного му…
	"уборка, стирка, домработница, помощница по хозяйс…
	"медицинская документация деловое общение медицинс…
	…
	"работа в команде управление курьерской службой ку…
	"парковочная система бесконфликтность работа в ком…
	null
	"работа в команде быстро учусь желание учиться жел…
	null
	null
	"приемка груза пользователь пк учет товаров"
	"работа с кассой пользователь пк работа в команде …
	"пользователь пк умение работать в команде учет то…
	"желание работать и зарабатывать высокая энергично…
	"контроль исправности оборудования инженерные сист…
	"предпечатная подготовка статистический анализ"
	"1с: то

In [16]:
vacancies

vacancy_id,name,company.id,description,keySkills.keySkill,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience
str,str,str,str,str,i64,i64,str,str,str,str,str,str
"""v_862116""","""Смотритель муз…","""c_162972""","""обязанности о…","""пользователь п…",16500,16500,"""RUR""","""a_4761""","""ar_33""","""full""","""fullDay""","""noExperience"""
"""v_288642""","""Ведущий менедж…","""c_208672""","""возможно наша …","""активные прода…",50000,50000,"""RUR""","""a_744""","""ar_2""","""full""","""fullDay""","""noExperience"""
"""v_1840054""","""Бухгалтер (по …","""c_198109""","""обязанности р…",null,50000,65000,"""RUR""","""a_6223""","""ar_78""","""full""","""fullDay""","""between3And6"""
"""v_2346232""","""Пекарь (Токсов…","""c_6137""","""для каждого кт…",null,38500,42000,"""RUR""","""a_4795""","""ar_51""","""full""","""fullDay""","""noExperience"""
"""v_312507""","""Торговый предс…","""c_206699""","""компания тд ол…","""продуктивность…",60000,60000,"""RUR""","""a_6837""","""ar_4""","""full""","""fullDay""","""between1And3"""
"""v_2323706""","""Менеджер по ка…","""c_244141""","""x5 ready food …","""контроль качес…",88000,88000,"""RUR""","""a_5920""","""ar_64""","""full""","""fullDay""","""between1And3"""
"""v_704266""","""Менеджер по пр…","""c_192038""","""если ты ищешь …","""активные прода…",30000,100000,"""RUR""","""a_5398""","""ar_71""","""full""","""fullDay""","""between1And3"""
"""v_525733""","""Врач по ЛФК""","""c_5101""","""обязанности в…",null,1,1,"""RUR""","""a_5043""","""ar_96""","""full""","""fullDay""","""between3And6"""
"""v_2091086""","""Монтажник СТиЖ…","""c_230890""","""строительная к…",null,100000,100000,"""RUR""","""a_500""","""ar_36""","""full""","""flyInFlyOut""","""between1And3"""


In [20]:
vacancies.filter(pl.col("compensation.currencyCode") == 'UZS')

vacancy_id,name,company.id,description,keySkills.keySkill,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience
str,str,str,str,str,i64,i64,str,str,str,str,str,str
"""v_1222387""","""Специалист слу…","""c_160874""","""пользователи я…",null,5530000,5530000,"""UZS""","""a_3964""",null,"""full""","""remote""","""noExperience"""
"""v_2337785""","""Менеджер по lu…","""c_112093""","""в компании bor…",null,8000000,8000000,"""UZS""","""a_3964""",null,"""full""","""fullDay""","""between3And6"""
"""v_446072""","""Региональный м…","""c_42112""","""обязанности п…",null,8000000,10000000,"""UZS""","""a_3964""",null,"""full""","""fullDay""","""between3And6"""
"""v_540297""","""Ведущий инжене…","""c_173551""","""обязанности р…","""разработка про…",8000000,8000000,"""UZS""","""a_3964""",null,"""full""","""fullDay""","""between3And6"""
"""v_875594""","""Специалист слу…","""c_160874""","""пользователи я…",null,5530000,5530000,"""UZS""","""a_6221""",null,"""full""","""remote""","""noExperience"""
"""v_440966""","""Промоутер-конс…","""c_146958""","""обязанности п…",null,2500000,8700000,"""UZS""","""a_3964""",null,"""full""","""fullDay""","""noExperience"""
"""v_1594879""","""Офис-менеджер""","""c_39683""","""в новый соврем…","""розничная торг…",5000000,5000000,"""UZS""","""a_3964""",null,"""full""","""fullDay""","""between1And3"""
"""v_2220635""","""Менеджер по пр…","""c_243045""","""яндекседа – эт…","""деловое общени…",1,7200000,"""UZS""","""a_5141""",null,"""full""","""remote""","""between1And3"""
"""v_849781""","""Менеджер по ра…","""c_104754""","""компания vian …","""клиентоориенти…",6500000,6500000,"""UZS""","""a_3964""",null,"""full""","""fullDay""","""between1And3"""


Нужно свести все валюты к одной или исключить вакансии с зарплатой не в рублях!

In [21]:
vacancies.select(['compensation.currencyCode']).unique()

compensation.currencyCode
str
"""BYR"""
"""UAH"""
"""AZN"""
"""KGS"""
"""EUR"""
"""RUR"""
"""KZT"""
"""UZS"""
"""USD"""


In [22]:
currency_rates = {"UZS": 0.007269,
                  "KGS": 1.01,
                  "USD": 89.69,
                  "GEL": 33.3,
                  "BYR": 28.23,
                  "AZN": 52.76,
                  "KZT": 0.197708,
                  "UAH": 2.36,
                  "RUR": 1.0,
                  "EUR": 99.19
                  }

Преобразуем суммы зарплат в соответствии с курсом ЦБ (на 09.01.2023)

In [23]:
vacancies = vacancies.rename({"compensation.currencyCode": "currencyRate", "compensation.from": "compensation_from", "compensation.to": "compensation_to"})

#vacancies = vacancies.with_columns(currencyRate=pl.col("currencyRate").replace(currency_rates, default=0.0))   # для версии 0.20
vacancies = vacancies.with_columns(currencyRate=pl.col("currencyRate").map_dict(currency_rates, default=0.0))   # для версии 0.19


vacancies = vacancies.with_columns(compensation_from = pl.col('compensation_from')*pl.col('currencyRate'))
vacancies = vacancies.with_columns(compensation_to = pl.col('compensation_to')*pl.col('currencyRate'))

In [24]:
vacancies.filter(pl.col("vacancy_id") == 'v_446072')  # посмотрим что получилось для вакансии с зарплатой в UZS

vacancy_id,name,company.id,description,keySkills.keySkill,compensation_from,compensation_to,currencyRate,area.id,area.regionId,employment,workSchedule,workExperience
str,str,str,str,str,f64,f64,f64,str,str,str,str,str
"""v_446072""","""Региональный м…","""c_42112""","""обязанности п…",null,58152.0,72690.0,0.007269,"""a_3964""",null,"""full""","""fullDay""","""between3And6"""


In [25]:
vacancies = vacancies.with_columns(salary_from=(pl.col("compensation_from").log10()*10).floor().cast(pl.Int64).cast(pl.Utf8)                 # Переведем зарплату в символьный вид
                                   ).with_columns(salary_to=(pl.col("compensation_to").log10()*10).floor().cast(pl.Int64).cast(pl.Utf8))     # потому что SASRecF не умеет с float

In [19]:
vacancies

vacancy_id,name,company.id,description,keySkills.keySkill,compensation_from,compensation_to,currencyRate,area.id,area.regionId,employment,workSchedule,workExperience,salary_from,salary_to
str,str,str,str,str,f64,f64,f64,str,str,str,str,str,str,str
"""v_862116""","""Смотритель муз…","""c_162972""","""обязанности о…","""пользователь п…",16500.0,16500.0,1.0,"""a_4761""","""ar_33""","""full""","""fullDay""","""noExperience""","""42""","""42"""
"""v_288642""","""Ведущий менедж…","""c_208672""","""возможно наша …","""активные прода…",50000.0,50000.0,1.0,"""a_744""","""ar_2""","""full""","""fullDay""","""noExperience""","""46""","""46"""
"""v_1840054""","""Бухгалтер (по …","""c_198109""","""обязанности р…",null,50000.0,65000.0,1.0,"""a_6223""","""ar_78""","""full""","""fullDay""","""between3And6""","""46""","""48"""
"""v_2346232""","""Пекарь (Токсов…","""c_6137""","""для каждого кт…",null,38500.0,42000.0,1.0,"""a_4795""","""ar_51""","""full""","""fullDay""","""noExperience""","""45""","""46"""
"""v_312507""","""Торговый предс…","""c_206699""","""компания тд ол…","""продуктивность…",60000.0,60000.0,1.0,"""a_6837""","""ar_4""","""full""","""fullDay""","""between1And3""","""47""","""47"""
"""v_2323706""","""Менеджер по ка…","""c_244141""","""x5 ready food …","""контроль качес…",88000.0,88000.0,1.0,"""a_5920""","""ar_64""","""full""","""fullDay""","""between1And3""","""49""","""49"""
"""v_704266""","""Менеджер по пр…","""c_192038""","""если ты ищешь …","""активные прода…",30000.0,100000.0,1.0,"""a_5398""","""ar_71""","""full""","""fullDay""","""between1And3""","""44""","""50"""
"""v_525733""","""Врач по ЛФК""","""c_5101""","""обязанности в…",null,1.0,1.0,1.0,"""a_5043""","""ar_96""","""full""","""fullDay""","""between3And6""","""0""","""0"""
"""v_2091086""","""Монтажник СТиЖ…","""c_230890""","""строительная к…",null,100000.0,100000.0,1.0,"""a_500""","""ar_36""","""full""","""flyInFlyOut""","""between1And3""","""50""","""50"""


In [26]:
vacancies.select(['employment']).unique()

employment
str
"""part"""
"""volunteer"""
"""project"""
"""full"""
"""probation"""


# SASRec

https://recbole.io/docs/user_guide/data_intro.html

https://recbole.io/docs/user_guide/data/atomic_files.html

https://recbole.io/docs/user_guide/usage/use_modules.html

https://www.recbole.io/docs/user_guide/model/sequential/sasrec.html

In [38]:
'''
unique_users = train['user_id'].unique().to_list()
unique_vacancies = train['vacancy_id'].explode().unique().to_list()

user2idx = {user_id: idx for idx, user_id in enumerate(unique_users)}
vac2idx = {vac_id: idx for idx, vac_id in enumerate(unique_vacancies)}

idx2vac = {idx: vac_id for vac_id, idx in vac2idx.items()}
'''

action_weights = {
    1: 4.0,
    2: 1.0,
    3: 2.0
}

pairs = train.select(['user_id', 'vacancy_id', 'action_type', 'action_dt']).explode(['vacancy_id', 'action_type', 'action_dt'])

In [39]:
pairs

user_id,vacancy_id,action_type,action_dt
str,str,i64,datetime[ns]
"""u_1057881""","""v_665861""",2,2023-11-01 00:23:51.452
"""u_1036784""","""v_2594840""",2,2023-11-01 00:52:34.023
"""u_786220""","""v_1473781""",1,2023-11-01 00:58:20.793
"""u_786220""","""v_1622905""",2,2023-11-01 01:17:04.774
"""u_786220""","""v_1621959""",2,2023-11-01 00:54:00.360
"""u_786220""","""v_2289180""",2,2023-11-01 01:16:05.415
"""u_786220""","""v_1473781""",2,2023-11-01 00:55:32.149
"""u_786220""","""v_1719419""",2,2023-11-01 01:09:54.498
"""u_786220""","""v_2654062""",2,2023-11-01 01:00:50.537


## Теперь для ALS подготовим матрицу

In [40]:
unique_users = train['user_id'].unique().to_list()
unique_vacancies = train['vacancy_id'].explode().unique().to_list()

user2idx = {user_id: idx for idx, user_id in enumerate(unique_users)}
vac2idx = {vac_id: idx for idx, vac_id in enumerate(unique_vacancies)}

idx2vac = {idx: vac_id for vac_id, idx in vac2idx.items()}
action_weights = {
    1: 4.0,
    2: 1.0,
    3: 2.0
}

users_list = pairs['user_id'].map_dict(user2idx, default=None).to_numpy()     # это для версии 0.19, для 0.20 надо .replace
vacancies_list = pairs['vacancy_id'].map_dict(vac2idx, default=None).to_numpy()
preferences = pairs['action_type'].map_dict(action_weights, default=None).to_numpy()

In [ ]:
# это для версии 0.20
#users_list = pairs['user_id'].replace(user2idx, default=None).to_numpy()
#vacancies_list = pairs['vacancy_id'].replace(vac2idx, default=None).to_numpy()
#preferences = pairs['action_type'].replace(action_weights, default=None).to_numpy()

In [41]:
uv_mat = csr_matrix((preferences, (users_list, vacancies_list)))   # (data, (row_ind, col_ind)

### а результат предобработки юзеров и вакансий запишем в датасет формата Recbole

In [35]:
def conv_date(source_date):
    return datetime.datetime.timestamp(source_date)

In [36]:
if not os.path.exists(DATASET_NAME):
    os.mkdir(DATASET_NAME)

In [34]:
# для версии 0.19

p = pairs.with_columns(action_dt=pl.col("action_dt").map_elements(conv_date)
                   ).with_columns(action_type=pl.col("action_type").map_dict(action_weights, default=0.0)
                                  ).rename({"user_id": "user_id:token", "vacancy_id": "vacancy_id:token", "action_type": "action_type:float", "action_dt": "timestamp:float"}
                                  ).write_csv("./" + DATASET_NAME + "/hh_recsys.inter", separator="\t")


In [ ]:
# для версии 0.20

pairs.with_columns(action_dt=pl.col("action_dt").map_elements(conv_date)
                   ).with_columns(action_type=pl.col("action_type").replace(action_weights, default=0.0)
                                  ).rename({"user_id": "user_id:token", "vacancy_id": "vacancy_id:token", "action_type": "action_type:float", "action_dt": "timestamp:float"}
                                           ).write_csv("./" + DATASET_NAME + "/hh_recsys.inter", include_header=True, separator="\t")

In [35]:
# для версии 0.19

vacancies.rename({"vacancy_id": "vacancy_id:token", "name": "name:token_seq", "company.id": "company_id:token", "description": "description:token_seq", 
                                 "keySkills.keySkill": "keySkills:token_seq", "compensation_from": "compensation_from:float", "compensation_to": "compensation_to:float", 
                                 "currencyRate": "currencyRate:float", "area.id": "area_id:token", "area.regionId": "area_regionId:token", 
                                 "employment": "employment:token", "workSchedule": "workSchedule:token", "workExperience": "workExperience:token",
                                 "salary_from": "salary_from:token", "salary_to": "salary_to:token"}                                 
                                 ).write_csv("./" + DATASET_NAME + "/hh_recsys.item", separator="\t")

In [ ]:
# для версии 0.20

vacancies.with_columns(pl.col("keySkills.keySkill").cast(pl.List(pl.Utf8)).list.join(", ")
                       ).rename({"vacancy_id": "vacancy_id:token", "name": "name:token_seq", "company.id": "company_id:token", "description": "description:token_seq", 
                                 "keySkills.keySkill": "keySkills:token_seq", "compensation_from": "compensation_from:float", "compensation_to": "compensation_to:float", 
                                 "currencyRate": "currencyRate:float", "area.id": "area_id:token", "area.regionId": "area_regionId:token", 
                                 "employment": "employment:token", "workSchedule": "workSchedule:token", "workExperience": "workExperience:token",
                                 "salary_from": "salary_from:token", "salary_to": "salary_to:token"}
                                 ).write_csv("./" + DATASET_NAME + "/hh_recsys.item", include_header=True, separator="\t")

Можно начинать с этой точки, если датасет уже сформирован

In [3]:
parameter_dict = {
    'data_path': './',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'vacancy_id',
    'RATING_FIELD': 'action_type',
    'TIME_FIELD': 'timestamp',
#    'MAX_ITEM_LIST_LENGTH': 30,  # для Caser    
#    'user_inter_num_interval': "[1,inf)",
#    'item_inter_num_interval': "[0,inf)",
    'user_inter_num_interval': "[10,inf)",
#    'seq_len': {'vacancy_id': 10}, # попробуем последовательности не больше 10. Так ли???
#    'MAX_ITEM_LIST_LENGTH': 10,    
    'load_col': {'inter': ['user_id', 'vacancy_id', 'action_type', 'timestamp'],
#                 'user': ['user_id'],
                 'item': ['vacancy_id', 'name', 'company_id', 'keySkills', 'salary_from', 'salary_to', 'area_id', 'area_regionId', 'employment', 'workSchedule', 'workExperience' ]},  # не все, но на большее не хватает памяти
#                 'item': ['vacancy_id', 'name', 'company_id', 'keySkills', 'compensation_from', 'area_regionId', 'employment', 'workSchedule', 'workExperience' ]},  # не все, но на большее не хватает памяти
#    'selected_features': ['name', 'company_id', 'keySkills', 'compensation_from', 'compensation_to', 'area_id', 'area_regionId', 'employment', 'workSchedule', 'workExperience' ],
#    'selected_features': ['name', 'company_id', 'keySkills'], - ок
#    'selected_features': ['compensation_from', 'compensation_to'], # mat1 and mat2 shapes cannot be multiplied (10x256 and 320x64)  или RuntimeError: torch.cat(): expected a non-empty list of Tensors
#    'selected_features': ['name', 'company_id', 'keySkills', 'area_id', 'area_regionId', 'employment', 'workSchedule', 'workExperience' ],
    'selected_features': ['name', 'company_id', 'keySkills', 'area_id', 'area_regionId', 'employment', 'workSchedule', 'workExperience', 'salary_from', 'salary_to'],
#    'numerical_features': ['compensation_from', 'compensation_to'],
#    'normalize_all': True,
    'neg_sampling': None,
    'train_neg_sample_args': None,
    'worker': 8,   # на vm66
    'train_batch_size': 1024,
    'eval_batch_size': 1024,
    'epochs': 5,
    'use_gpu': False,
#    'metrics': ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision'],
    'metrics': ['Recall', 'MRR'],
    'loss_type': 'CE',
#    'train_neg_sample_args': {'uniform': 1},  # для TransRec  
#    'loss_type': 'BPR',  # для FPMC
#    'train_neg_sample_args': {'distribution': 'uniform', 'sample_num': 1},  # для FPMC
    'reproducibility': False,   # для HRM или SHAN
    'topk': 100,
    'valid_metric': 'MRR@100',
    "stopping_step": 2, # останавливаем обучение если в течение этого кол-ва эпох не было улучшения валидационной метрики
    'hidden_size': 64,  # по дефолту 64
    'inner_size': 256,  # по дефолту 256
    'hidden_dropout_prob': 0.3,  # 0.5
    'attn_dropout_prob': 0.3,    # 0.5
    'seed': 42,
    'eval_args': {
        'split': {'RS': [10, 0, 0]},  # 8,1,1 ?
#        'split': {'RS': [0.95, 0.03, 0.02]},  # 8,1,1 ?
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'}  #  labeled ??
}

train_batch_size: 2048  по дефолту<br>
eval_batch_size: 4096

In [ ]:
config = Config(model='SASRec', dataset=DATASET_NAME, config_dict=parameter_dict)

In [ ]:
config = Config(model='SLIMElastic', dataset=DATASET_NAME, config_dict=parameter_dict)

In [ ]:
hh_recsys_dataset = create_dataset(config)

In [ ]:
hh_recsys_dataset # это 30 и 50, датасет объединенный с test

In [ ]:
# hh_recsys_dataset # это 10 и 15 и цепочки max 10, датасет объединенный с test

In [ ]:
# hh_recsys_dataset  # это 10 и 15

In [ ]:
# hh_recsys_dataset   # это полный

In [ ]:
# train_data, valid_data, test_data = data_preparation(config, hh_recsys_dataset)   # работает 8 минут

In [4]:
#run_recbole(model='SASRecF', dataset=DATASET_NAME, config_dict=parameter_dict)
run_recbole(model='SINE', dataset=DATASET_NAME, config_dict=parameter_dict)

06 Feb 09:10    INFO  ['/home/vm66/.local/lib/python3.9/site-packages/ipykernel_launcher.py', '--f=/home/vm66/.local/share/jupyter/runtime/kernel-v2-30242733iY1MzJsTy4m.json']
06 Feb 09:10    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = False
seed = 42
state = INFO
reproducibility = False
data_path = ./hh_recsys
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 5
train_batch_size = 1024
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 2
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [10, 0, 0]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = True
metrics = ['Recall', 'MRR

: 

Надо будет использовать full_sort_topk ?

как описано тут: https://recbole.io/docs/user_guide/usage/case_study.html  <br>
и тут: https://www.kaggle.com/code/astrung/recbole-lstm-sequential-for-recomendation-tutorial

Или методы модели https://recbole.io/docs/recbole/recbole.model.sequential_recommender.sasrec.html

In [ ]:
samples = pl.read_parquet("./hh_recsys_sample.pq")

In [ ]:
samples

в т.ч. можно использовать для загрузки при обрыве

In [1]:
!ls saved

ALS-200-fulldata.npz		  SASRec-Jan-04-2024_07-58-21.pth
ALS-200-fulldata.pkl		  SASRec-Jan-06-2024_02-07-14.pth
FPMC-Jan-14-2024_15-27-07.pth	  SASRec-Jan-16-2024_01-44-14.pth
ItemKNN-Jan-23-2024_15-30-10.pth  SHAN-Jan-10-2024_12-13-04.pth
ItemKNN-Jan-26-2024_15-20-50.pth  SINE-Jan-23-2024_18-05-07.pth
SASRecF-Jan-27-2024_15-13-17.pth


In [3]:
!ls models-gpu

Следующая ячейка выполняется 12 минут.

In [ ]:
model_cpu = torch.load('models-gpu/HRM-Jan-11-2024_18-18-49.pth', map_location=torch.device('cpu'))

In [ ]:
torch.save(model_cpu, "saved/HRM-Jan-11-2024_18-18-49.pth")

In [54]:

config, model, dataset, train_data, valid_data, test_data = load_data_and_model(
    model_file='saved/SASRecF-Jan-27-2024_15-13-17.pth',
)

01 Feb 17:10    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = False
seed = 42
state = INFO
reproducibility = True
data_path = ./hh_recsys
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 5
train_batch_size = 1024
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 2
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.95, 0.03, 0.02]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = True
metrics = ['Recall', 'MRR']
topk = [100]
valid_metric = MRR@100
valid_metric_bigger = True
eval_batch_size = 1024
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_sepa

In [ ]:
model_file='saved/HRM-Jan-11-2024_18-18-49.pth'

In [ ]:
import torch

In [ ]:
from recbole.utils import init_seed, init_logger, get_model
from logging import getLogger

In [ ]:
# то же самое что load_data_and_model, но вручную и с подменой device
checkpoint = torch.load(model_file)
config = checkpoint["config"]

config["use_gpu"] = False  #  вот и проверим
config["device"] = torch.device("cpu")

init_seed(config["seed"], config["reproducibility"])
init_logger(config)
logger = getLogger()
logger.info(config)

dataset = create_dataset(config)
logger.info(dataset)
train_data, valid_data, test_data = data_preparation(config, dataset)

init_seed(config["seed"], config["reproducibility"])
#model = get_model(config["model"])(config, train_data._dataset).to(config["device"])
model = get_model(config["model"])(config, train_data._dataset).to(torch.device("cpu"))
model.load_state_dict(checkpoint["state_dict"])
model.load_other_parameter(checkpoint.get("other_parameter"))

In [11]:
config['epochs']=5

In [12]:

# trainer loading and initialization
trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

# resume from break point
checkpoint_file = 'saved/SASRecF-Jan-27-2024_15-13-17.pth'
trainer.resume_checkpoint(checkpoint_file)

# model training
best_valid_score, best_valid_result = trainer.fit(train_data, valid_data, show_progress=True)


Evaluate   :  99%|███████████████████████████████████████████████▌| 367/370 [06:55<00:04,  1.54s/it]:  99%|███████████████████████████████████████████████▋| 368/370 [06:56<00:02,  1.40s/it]: 100%|███████████████████████████████████████████████▊| 369/370 [06:57<00:01,  1.33s/it]: 100%|████████████████████████████████████████████████| 370/370 [06:58<00:00,  1.21s/it]: 100%|████████████████████████████████████████████████| 370/370 [06:58<00:00,  1.13s/it]
28 Jan 23:03    INFO  epoch 4 evaluating [time: 420.02s, valid_score: 0.165300]
28 Jan 23:03    INFO  valid result: 
recall@100 : 0.5724    mrr@100 : 0.1653
28 Jan 23:03    INFO  Saving current: saved/SASRecF-Jan-27-2024_15-13-17.pth


Исходные данные нам больше не нужны

In [41]:
import gc

del train
# del test  # нам еще понадобится
del vacancies

gc.collect()

318

#### Теперь обучим ALS как модель второго эшелона

In [42]:
als_model = implicit.als.AlternatingLeastSquares(
    factors=200,   # 150
    random_state=RANDOM_STATE,
    iterations=100,
    alpha=3.0,
    calculate_training_loss=True,
    regularization=0.001,
    num_threads=8
)
als_model.fit(uv_mat)

  0%|          | 0/100 [00:00<?, ?it/s]

In [46]:
als_model.save("saved/ALS-200-fulldata.npz")

In [45]:
with open('saved/ALS-200-fulldata.pkl', 'wb') as f:
    pickle.dump(als_model, f)

In [55]:
test = pl.read_parquet("./hh_recsys_test_hh.pq", low_memory=True)
#vacancies = pl.read_parquet("./hh_recsys_vacancies.pq")

In [48]:
test_users = test['user_id'].to_list()
test_vacancies = test.select(pl.col('vacancy_id').list.unique(maintain_order=True))['vacancy_id'].to_list()

In [49]:
# Предсказание чисто ALS

predictions = []

for user, vacs in tqdm(zip(test_users, test_vacancies), total=len(test_users)):
    if user not in user2idx:
        predictions.append(vacs)
        continue
    cuser = user2idx[user]

    recommendations = als_model.recommend(cuser, uv_mat[cuser], N=N_PREDICTIONS, filter_already_liked_items=False)[0]
    recommendations = [idx2vac[cv] for cv in recommendations]
    predictions.append(recommendations)

100%|██████████| 83189/83189 [54:20<00:00, 25.51it/s]  


In [52]:
#test = test.with_columns(pl.lit(predictions).alias('predictions'))
test = test.with_columns(pl.lit(pl.Series(predictions)).alias('predictions'))
test.select(['user_id', 'session_id', 'predictions']).write_parquet('als200-full_submission.pq')

In [ ]:
cuser = dataset.token2id(dataset.uid_field, 'u_186164') 
cuser

In [ ]:
scores, topk_iid_list = full_sort_topk([cuser], model, test_data, k=10, device=config['device'])

In [ ]:
topk_iid_list[-1]

In [ ]:
last_topk_iid_list = topk_iid_list[-1]

In [ ]:
recommendations = dataset.id2token(dataset.iid_field, last_topk_iid_list.cpu()).tolist()

In [ ]:
recommendations

In [ ]:
bad_list = ['u_186164']

Это для моделей из implicit

https://benfred.github.io/implicit/api/models/recommender_base.html#implicit.recommender_base.RecommenderBase

М.б. model.recommend(userid, user_items, N=10, filter_already_liked_items=True, filter_items=None, recalculate_user=False, items=None)

recalculate_user=True, filter_already_liked_items=False (?)

In [16]:
predictions = []
num_not_found = 0
zero_topk = 0

for user, vacs in tqdm(zip(test_users, test_vacancies), total=len(test_users)):
#    print("Try: ", user)
    if user not in dataset.field2token_id[dataset.uid_field]:   # либо в тесте есть такой юзер, которого не было в трейне, либо в трейне он отсеялся по порогу (?)
        predictions.append(vacs)
        num_not_found += 1
#        print("Not found user: ", user)
        
        continue
    
    cuser = dataset.token2id(dataset.uid_field, user)   # получили id очередного юзера по его имени
    

    _, topk_iid_list = full_sort_topk([cuser], model, test_data, k=N_PREDICTIONS, device=config['device'])
#    _, topk_iid_list = full_sort_topk([cuser], model, test_data, k=N_PREDICTIONS, device='cpu')
    if len(topk_iid_list) == 0:
#        print("Zero topk for user ", user)
        predictions.append(vacs)
        zero_topk += 1
        continue
    
    last_topk_iid_list = topk_iid_list[-1]
    recommendations = dataset.id2token(dataset.iid_field, last_topk_iid_list.cpu()).tolist()

    predictions.append(recommendations)

print("Not found: ", num_not_found)
print("Zero topk: ", zero_topk)


  0%|          | 0/83189 [00:00<?, ?it/s]

Not found:  30906
Zero topk:  0


Для юзеров, которые есть в тесте, но не попали в трейн, можно:
- тупо брать те вакансии что они смотрели
- использовать ALS
- использовать какую-нибудь другую модель
- как-то находить самую похожую последовательность вакансий и предсказывать для такого юзера?  или искать такую в тесте?.. ???

Как придумать предсказание на основе последовательности для любого юзера, независимо от его id?..


In [27]:
test = test.with_columns(pl.lit(pl.Series(predictions)).alias('predictions'))
test.select(['user_id', 'session_id', 'predictions']).write_parquet('sasrecf-10-15_good-dataset_submission.pq')

AttributeError: 'dict' object has no attribute 'with_columns'

In [ ]:
score, topk_iid_list = full_sort_topk([1], model, test_data, k=6, device=config['device'])
last_topk_iid_list = topk_iid_list[-1]
external_item_list = dataset.id2token(dataset.iid_field, last_topk_iid_list.cpu()).tolist()
external_item_list

In [ ]:
topk_iid_list.shape

In [ ]:
score

In [ ]:
dataset.uid_field

In [ ]:
type(dataset)

In [ ]:
dir(dataset)

In [ ]:
dataset.history_item_matrix(value_field='1')

In [ ]:
dataset[dataset.uid_field]

In [ ]:
index = np.isin(dataset[dataset.uid_field].numpy(), [1])
input_interaction = dataset[index]
input_interaction

In [ ]:
input_features = test_data.dataset[np.isin(test_data.dataset[test_data.dataset.uid_field].numpy(), [2])]
input_features

## Попробуем предсказания последнего item-а

In my previous notebook about sequential model with Recbole, someone asked me about the mechanism of test data when using full_sort_topk for prediction submitted recommendation in this comment and this comment, and they have a doubt about whether we are using all items for getting final recommendation. Most of people has 2 questions about using full_sort_topk with test data:

Do items in test data are used as input features for getting recommendation ?
If test data is necessary for getting recommendation in Recbole API, how can we get recommendation without splitting into train/test data?
In this notebook i will answer all questions:

Yes. In sequential models, items in test data is used as input features, but not last items. As a example, if user X have 3 items in test data(A, B, C) and 5 items in train data(a,b,c,d,e), test data will generate 3 sample rows for evaluating performance on user X:
Row 1: Input features: a,b,c,d,e,0,0. Output features: A. 0 is a pad item
Row 2: Input features: a,b,c,d,e,A,0. Output features: B.
Row 3: Input features: a,b,c,d,e,A,B. Output features: C.
In my previous notebook, i use last row result as recommendation, so we still using nearly all of items as input for recommendation, except last item(item C). Our recommendation in previous notebooks may be not perfect, but it is simple as a tutorial for anyone want to start.

Note: This mechanism is only for sequential model in recbole. For other types of model, it isn't correct - it won't use items in test data for getting recommendation. If you have requests for explaining for other model, please upvote and comment. I will explain it in other notebook

In first session of this notebook, i will dig into test data to prove this conclusion.

Yes, we can get recommendation by using all of items as input features, without splitting train/test. In order to do this, you need to modify recbole code:
Fist, you copy last row in dataset(input features have all items, except last one), then add last item into input features.
Then you predict directly from model api, without using full_sort_score or full_sort_topk
In second session of this notebook, i will show you how to do that.

I. How test items are used in test data.
For each item in test data, it will be generated as a sample row. As a example, if user X have 3 items in test data(A, B, C) and 5 items in train data(a,b,c,d,e), test data will generate 3 sample rows for evaluating performance on user X:

Row 1: Input features: a,b,c,d,e,0,0. Output label: A. 0 is a pad item
Row 2: Input features: a,b,c,d,e,A,0. Output label: B.
Row 3: Input features: a,b,c,d,e,A,B. Output label: C.
For proving it, we will create a dataset, then extract input features and label in test data.

In [7]:
test_data.dataset.uid_field

'user_id'

In [8]:
test_data.dataset.token2id(test_data.dataset.uid_field, 'u_786220')

1

In [9]:
dataset.iid_field

'vacancy_id'

In [119]:
test = pl.read_parquet("./hh_recsys_test_hh.pq", low_memory=True)

In [120]:
test 

user_id,session_id,vacancy_id,action_type,action_dt
str,str,list[str],list[i64],list[datetime[ns]]
"""u_482520""","""s_25018731""","[""v_2597196"", ""v_1223061"", ""v_1223061""]","[2, 2, 1]","[2023-11-19 12:03:13.089, 2023-11-19 12:03:30.396, 2023-11-19 12:05:03.473]"
"""u_582132""","""s_481216""","[""v_470400"", ""v_470400"", ""v_1530783""]","[2, 1, 2]","[2023-11-21 15:39:47.981, 2023-11-21 15:43:57.620, 2023-11-21 15:44:37.982]"
"""u_212584""","""s_16918781""","[""v_1572055"", ""v_1572055"", … ""v_1329494""]","[2, 3, … 1]","[2023-11-16 08:41:47.031, 2023-11-16 08:45:21.403, … 2023-11-16 09:05:27.379]"
"""u_425177""","""s_17505104""","[""v_1375331"", ""v_1922852""]","[2, 2]","[2023-11-17 12:42:18.513, 2023-11-17 12:43:52.744]"
"""u_700997""","""s_15528830""","[""v_2152997"", ""v_2152997"", ""v_1217630""]","[2, 1, 2]","[2023-11-16 17:22:53.530, 2023-11-16 17:23:19.885, 2023-11-16 17:23:33.472]"
"""u_687201""","""s_11187326""","[""v_2602959"", ""v_549337"", … ""v_212060""]","[2, 2, … 1]","[2023-11-21 14:41:30.546, 2023-11-21 15:00:25.506, … 2023-11-21 15:24:56.271]"
"""u_573219""","""s_27394215""","[""v_1836184"", ""v_2733253"", … ""v_2297339""]","[1, 2, … 2]","[2023-11-21 18:48:43.424, 2023-11-21 18:49:12.552, … 2023-11-21 19:48:55.704]"
"""u_1082782""","""s_30341164""","[""v_1026036"", ""v_1026036"", … ""v_206700""]","[2, 1, … 2]","[2023-11-17 02:54:19.561, 2023-11-17 02:54:50.486, … 2023-11-17 03:21:22.008]"
"""u_81444""","""s_1909964""","[""v_2497384"", ""v_1480870"", … ""v_1500697""]","[2, 2, … 1]","[2023-11-15 12:38:34.579, 2023-11-15 13:03:41.360, … 2023-11-15 13:05:15.916]"


In [12]:
last_item_ids = test.filter(pl.col(test_data.dataset.uid_field) == 'u_1084868').select(pl.col("vacancy_id")).item().to_list()
last_item_ids

['v_395706', 'v_2405231']

Recbole use an internal ids for identify user_id and item_id, so let convert this user_id and his items into internal ids.

In [13]:
print(dataset.token2id(dataset.iid_field, last_item_ids))

[178271 113615]


In [96]:
dataset.token2id(dataset.uid_field, 'u_885219')

289282

In [14]:
test_data.dataset.uid_field

'user_id'

In [15]:
test_data.dataset.item_feat

The batch_size of interaction: 214819
    vacancy_id, torch.Size([214819]), cpu, torch.int64
    name, torch.Size([214819, 26]), cpu, torch.int64
    company_id, torch.Size([214819]), cpu, torch.int64
    keySkills, torch.Size([214819, 141]), cpu, torch.int64
    area_id, torch.Size([214819]), cpu, torch.int64
    area_regionId, torch.Size([214819]), cpu, torch.int64
    employment, torch.Size([214819]), cpu, torch.int64
    workSchedule, torch.Size([214819]), cpu, torch.int64
    workExperience, torch.Size([214819]), cpu, torch.int64
    salary_from, torch.Size([214819]), cpu, torch.int64
    salary_to, torch.Size([214819]), cpu, torch.int64


In [16]:
test_data.dataset.feat_name_list

['inter_feat', 'item_feat']

In [17]:
test_data.dataset.inter_feat

The batch_size of interaction: 327064
    user_id, torch.Size([327064]), cpu, torch.int64
    vacancy_id, torch.Size([327064]), cpu, torch.int64
    action_type, torch.Size([327064]), cpu, torch.float32
    timestamp, torch.Size([327064]), cpu, torch.float32
    item_length, torch.Size([327064]), cpu, torch.int64
    vacancy_id_list, torch.Size([327064, 10]), cpu, torch.int64
    action_type_list, torch.Size([327064, 10]), cpu, torch.float32
    timestamp_list, torch.Size([327064, 10]), cpu, torch.float32


In [8]:
test_data.dataset.inter_feat['vacancy_id_list']

tensor([[ 88040, 161035,  28595,  ...,  32333,  32333, 108429],
        [161035,  28595,   7463,  ...,  32333, 108429, 213577],
        [ 28595,   7463, 172372,  ..., 108429, 213577,  39711],
        ...,
        [  7093,  85906, 135675,  ...,  13031, 214558, 126895],
        [   796,  12320,  46415,  ...,   5336,  97367,  12337],
        [ 24799,  65934,  51421,  ...,  93796,  54235,   7856]])

In [9]:
test_data.dataset.inter_feat['user_id']  # это все юзеры, по которым было взаимодействие (которые есть в датасете - !). Могут быть дубли

tensor([  1056,   1056,   1056,  ..., 285099, 290799, 289382])

In [20]:
test_data.dataset.inter_feat['user_id'].shape

torch.Size([327064])

In [147]:
np.unique(test_data.dataset.inter_feat['user_id'])

array([     1,      2,      3, ..., 291496, 291497, 291498])

In [149]:
np.unique(test_data.dataset.inter_feat[test_data.dataset.uid_field])

array([     1,      2,      3, ..., 291496, 291497, 291498])

Now let extract input features and labels in our test data. 

In [10]:
type(test_data.dataset.inter_feat[test_data.dataset.uid_field])

torch.Tensor

In [11]:
input_features = test_data.dataset[np.isin(test_data.dataset.inter_feat[test_data.dataset.uid_field].numpy(), [40729])]
#%time
#input_features = test_data.dataset[torch.isin(test_data.dataset.inter_feat[test_data.dataset.uid_field], torch.tensor([40729])).numpy()]    # а так через Pytorch

item_id in above interaction is used as label item
item_id_list in above interaction is used as feature items

In [12]:
input_features

The batch_size of interaction: 1
    user_id, torch.Size([1]), cpu, torch.int64
    vacancy_id, torch.Size([1]), cpu, torch.int64
    action_type, torch.Size([1]), cpu, torch.float32
    timestamp, torch.Size([1]), cpu, torch.float32
    item_length, torch.Size([1]), cpu, torch.int64
    vacancy_id_list, torch.Size([1, 10]), cpu, torch.int64
    action_type_list, torch.Size([1, 10]), cpu, torch.float32
    timestamp_list, torch.Size([1, 10]), cpu, torch.float32
    name, torch.Size([1, 26]), cpu, torch.int64
    company_id, torch.Size([1]), cpu, torch.int64
    keySkills, torch.Size([1, 141]), cpu, torch.int64
    area_id, torch.Size([1]), cpu, torch.int64
    area_regionId, torch.Size([1]), cpu, torch.int64
    employment, torch.Size([1]), cpu, torch.int64
    workSchedule, torch.Size([1]), cpu, torch.int64
    workExperience, torch.Size([1]), cpu, torch.int64
    salary_from, torch.Size([1]), cpu, torch.int64
    salary_to, torch.Size([1]), cpu, torch.int64


In [157]:
input_features['vacancy_id'].item()

213763

In [31]:
print("test label: " + str(input_features['vacancy_id']))
print("last 10 items from origin dataset: " + str(dataset.token2id(dataset.iid_field, last_item_ids)))

test label: tensor([213763])
last 10 items from origin dataset: [178271 113615]


In [32]:
input_features['vacancy_id_list'].shape

torch.Size([1, 10])

For 1st row, it uses all items in training as input features.
For 2nd row, it uses all items in training + first label as input features
For 3rd row, it uses all items in training + first label + second label as input features
...
For last row, it uses all items except last item as input features.

so we are missing information from last item.

So now let fix it- find a new way for using all items

We have seen that last row is missing only last item, so fixxing ideal is simple now:

copy last row, add last item into it as a new interation(a row in test dataset)
make prediction with new interation
So now let train a dummy model for testing it

Our sequence items is always have fix length(50). So if we have more than 50 items, we need to drop earlier items, and if there are less than 50 items, we need to add a padding(0) into input item features. As example:

If last row input = [3, 4, 7,..., 20, 0, 0 ,0] (47 items < 50 item, so we have padding), after adding id=30, we will have input = [3, 4, 7,..., 20, 30, 0 ,0] Now our sequence lenght = 48 items.
If last row input = [3, 4, 7,..., 20, 9, 10 ,12] (50 items), after adding id=30, we will have input = [4, 7,..., 9, 10, 12, 30] (drop first item and add last item).Now our sequence lenght still = 50 items.
Now let implement it.

First let extract last row from all interation when internal_user_id = 2

In [34]:
index = np.isin(dataset.inter_feat[dataset.uid_field].numpy(), [40729])  # 291496 40729  # 11 + 64   # np.isin

In [37]:
index2 = np.argwhere(index)

In [40]:
input_interaction = dataset[np.argwhere(index)]

In [41]:
input_interaction

The batch_size of interaction: 64
    user_id, torch.Size([64, 1]), cpu, torch.int64
    vacancy_id, torch.Size([64, 1]), cpu, torch.int64
    action_type, torch.Size([64, 1]), cpu, torch.float32
    timestamp, torch.Size([64, 1]), cpu, torch.float32
    item_length, torch.Size([64, 1]), cpu, torch.int64
    vacancy_id_list, torch.Size([64, 1, 10]), cpu, torch.int64
    action_type_list, torch.Size([64, 1, 10]), cpu, torch.float32
    timestamp_list, torch.Size([64, 1, 10]), cpu, torch.float32
    name, torch.Size([64, 1, 26]), cpu, torch.int64
    company_id, torch.Size([64, 1]), cpu, torch.int64
    keySkills, torch.Size([64, 1, 141]), cpu, torch.int64
    area_id, torch.Size([64, 1]), cpu, torch.int64
    area_regionId, torch.Size([64, 1]), cpu, torch.int64
    employment, torch.Size([64, 1]), cpu, torch.int64
    workSchedule, torch.Size([64, 1]), cpu, torch.int64
    workExperience, torch.Size([64, 1]), cpu, torch.int64
    salary_from, torch.Size([64, 1]), cpu, torch.int64
    sa

In [97]:
dataset[289282]

The batch_size of interaction: 10
    user_id, torch.Size([]), cpu, torch.int64
    vacancy_id, torch.Size([]), cpu, torch.int64
    action_type, torch.Size([]), cpu, torch.float32
    timestamp, torch.Size([]), cpu, torch.float32
    item_length, torch.Size([]), cpu, torch.int64
    vacancy_id_list, torch.Size([10]), cpu, torch.int64
    action_type_list, torch.Size([10]), cpu, torch.float32
    timestamp_list, torch.Size([10]), cpu, torch.float32
    name, torch.Size([26]), cpu, torch.int64
    company_id, torch.Size([]), cpu, torch.int64
    keySkills, torch.Size([141]), cpu, torch.int64
    area_id, torch.Size([]), cpu, torch.int64
    area_regionId, torch.Size([]), cpu, torch.int64
    employment, torch.Size([]), cpu, torch.int64
    workSchedule, torch.Size([]), cpu, torch.int64
    workExperience, torch.Size([]), cpu, torch.int64
    salary_from, torch.Size([]), cpu, torch.int64
    salary_to, torch.Size([]), cpu, torch.int64


In [99]:
index = np.isin(dataset.inter_feat[dataset.uid_field].numpy(), [289282])   # 40729

In [89]:
np.nonzero(index)[0]

array([ 1157120,  1359025,  3304546,  3304547,  4353641,  4353642,
        4373144,  4389929,  4389930,  4395415,  4395416,  4395417,
        4395418,  4395419,  4395420,  4395421,  4398292,  4398293,
        4398294,  4400933,  4403554,  4403555,  4406191,  4406192,
        4406193,  4406194,  4406195,  4406196,  4406197,  4406198,
        4406199,  4406200,  4406201,  4406202,  4406203,  4406204,
        4406205,  4406206,  4406207,  4406208,  4406209,  4406210,
        4406211,  4406212,  4411451,  4414032,  4419270,  4419271,
        4434959,  4434960,  4437538,  6483271,  6485853,  6488394,
        6488395,  6488396,  6488397,  6488398,  6490957,  9950239,
       10853905, 10855034, 10855703, 10855787])

Now let add last item into sequences, and make new interaction. We also need to edit sequence lenght (without padding)

In [103]:
import torch
from recbole.data.interaction import Interaction

def add_last_vacancy(old_interaction, last_vacancy_id, max_len=10):  # было max_len=50
    new_seq_vacancies = old_interaction['vacancy_id_list'][-1]
    if old_interaction['item_length'][-1].item() < max_len:
        new_seq_vacancies[input_interaction['item_length'][-1].item()] = last_vacancy_id
    else:
        new_seq_vacancies = torch.roll(new_seq_vacancies, -1)
        new_seq_vacancies[-1] = last_vacancy_id
    return new_seq_vacancies.view(1, len(new_seq_vacancies))

test = {
            'vacancy_id_list': add_last_vacancy(input_interaction, input_interaction['vacancy_id'][-1].item(), model.max_seq_length),
            'item_length': torch.tensor(
                [input_interaction['item_length'][-1].item() + 1
                 if input_interaction['item_length'][-1].item() < model.max_seq_length else model.max_seq_length])
        }
new_inter = Interaction(test)
new_inter

The batch_size of interaction: 1
    vacancy_id_list, torch.Size([1, 10]), cpu, torch.int64
    item_length, torch.Size([1]), cpu, torch.int64


In [108]:
new_inter['vacancy_id_list']

tensor([[ 84548,  84548, 104162, 104162, 201448,  25052,  24355,  25052,  25052,
          47335]])

Interaction for GRU4Rec model need to have only item_id_list and item_lenght. You can drop other key. If you want more information, you can check GRU4Rec code

Then we can apply the remaining prediction code from full_sort_scores

In [109]:
new_inter = new_inter.to(config['device'])
new_scores = model.full_sort_predict(new_inter)
new_scores = new_scores.view(-1, test_data.dataset.item_num)
new_scores[:, 0] = -np.inf  # set scores of [pad] to -inf

So now by combining all fragments,we have a new function for predicting with all item in dataset. You can use this custom code for all sequential model

Взято отсюда: https://www.kaggle.com/code/astrung/recbole-using-all-items-for-prediction

## Итоговый код для предсказания на всех взаимодействиях

In [56]:
def add_last_vacancy(old_interaction, last_vacancy_id, max_len=10):   # изначально max_len=50
    new_seq_vacancies = old_interaction['vacancy_id_list'][-1]
    if old_interaction['item_length'][-1].item() < max_len:
        new_seq_vacancies[old_interaction['item_length'][-1].item()] = last_vacancy_id
    else:
        new_seq_vacancies = torch.roll(new_seq_vacancies, -1)
        new_seq_vacancies[-1] = last_vacancy_id
    return new_seq_vacancies.view(1, len(new_seq_vacancies))

In [57]:
def predict_for_all_item(external_user_id, dataset, model):
    model.eval()
    with torch.no_grad():
        uid_series = dataset.token2id(dataset.uid_field, [external_user_id])
        index = np.isin(dataset.inter_feat[dataset.uid_field].numpy(), uid_series)
        input_interaction = dataset[np.nonzero(index)[0]]
#        input_interaction = dataset[index]  # 9 секунд на полном датасете
        test = {
            'vacancy_id_list': add_last_vacancy(input_interaction,
                                                input_interaction['vacancy_id'][-1].item(), model.max_seq_length),
            'item_length': torch.tensor(
                [input_interaction['item_length'][-1].item() + 1
                 if input_interaction['item_length'][-1].item() < model.max_seq_length else model.max_seq_length])
        }
        new_inter = Interaction(test)
        new_inter = new_inter.to(config['device'])
        new_scores = model.full_sort_predict(new_inter)
        new_scores = new_scores.view(-1, test_data.dataset.item_num)
        new_scores[:, 0] = -np.inf  # set scores of [pad] to -inf
    return torch.topk(new_scores, N_PREDICTIONS)

In [112]:
torch.topk(new_scores, N_PREDICTIONS)

torch.return_types.topk(
values=tensor([[14.3389, 14.0669, 14.0236, 13.8455, 13.6557, 13.5191, 13.3359, 13.2251,
         13.1959, 13.1597, 13.0343, 12.9661, 12.9437, 12.9436, 12.9337, 12.8954,
         12.8509, 12.8388, 12.8159, 12.7106, 12.7051, 12.6965, 12.6950, 12.6817,
         12.6732, 12.5698, 12.5396, 12.4783, 12.4731, 12.4493, 12.4251, 12.4190,
         12.3312, 12.3008, 12.2841, 12.2558, 12.2264, 12.2226, 12.1985, 12.1276,
         12.1253, 12.1151, 12.0830, 12.0668, 12.0460, 12.0228, 12.0220, 12.0140,
         12.0116, 11.9861, 11.9765, 11.9696, 11.9588, 11.8891, 11.8603, 11.8292,
         11.8292, 11.8161, 11.8052, 11.7602, 11.7092, 11.7017, 11.6783, 11.6643,
         11.6612, 11.6508, 11.5972, 11.5856, 11.5526, 11.5499, 11.5359, 11.5227,
         11.5183, 11.5083, 11.5080, 11.4895, 11.4775, 11.4706, 11.4326, 11.4318,
         11.4313, 11.4147, 11.4134, 11.4129, 11.4111, 11.4101, 11.4052, 11.3911,
         11.3791, 11.3750, 11.3683, 11.3590, 11.3580, 11.3021, 11.2924, 11.28

In [113]:
predict_for_all_item('u_885219', dataset, model) # we feed directly origin dataset, not train data or test data
# u_27420

torch.return_types.topk(
values=tensor([[14.3389, 14.0669, 14.0236, 13.8455, 13.6557, 13.5191, 13.3359, 13.2251,
         13.1959, 13.1597, 13.0343, 12.9661, 12.9437, 12.9436, 12.9337, 12.8954,
         12.8509, 12.8388, 12.8159, 12.7106, 12.7051, 12.6965, 12.6950, 12.6817,
         12.6732, 12.5698, 12.5396, 12.4783, 12.4731, 12.4493, 12.4251, 12.4190,
         12.3312, 12.3008, 12.2841, 12.2558, 12.2264, 12.2226, 12.1985, 12.1276,
         12.1253, 12.1151, 12.0830, 12.0668, 12.0460, 12.0228, 12.0220, 12.0140,
         12.0116, 11.9861, 11.9765, 11.9696, 11.9588, 11.8891, 11.8603, 11.8292,
         11.8292, 11.8161, 11.8052, 11.7602, 11.7092, 11.7017, 11.6783, 11.6643,
         11.6612, 11.6508, 11.5972, 11.5856, 11.5526, 11.5499, 11.5359, 11.5227,
         11.5183, 11.5083, 11.5080, 11.4895, 11.4775, 11.4706, 11.4326, 11.4318,
         11.4313, 11.4147, 11.4134, 11.4129, 11.4111, 11.4101, 11.4052, 11.3911,
         11.3791, 11.3750, 11.3683, 11.3590, 11.3580, 11.3021, 11.2924, 11.28

Можно делать так:

In [114]:
_, topk_iid_list = predict_for_all_item('u_885219', dataset, model)     # u_27420
last_topk_iid_list = topk_iid_list[-1]
external_item_list = dataset.id2token(dataset.iid_field, last_topk_iid_list.cpu()).tolist()

Congratulation !!!.Now you can use all data as train set, don't need for a test set, but still can predict directly from dataset without testset.Now let apply it into our previous notebook.

Сравним со старым способом:

In [82]:
cuser = dataset.token2id(dataset.uid_field, 'u_27420') 
scores, topk_iid_list = full_sort_topk([cuser], model, test_data, k=N_PREDICTIONS, device=config['device'])

In [83]:
topk_iid_list

tensor([[ 73927,  73929,  45371,  73924, 188568, 157228, 211232, 168561, 119417,
          85973,  51842, 109158, 210108,  73931, 114783, 199070, 155645, 201392,
         100142, 170783,  23956,  23952,  37543, 168560, 185014,  18950, 150418,
         155644, 133709,  45372, 159896, 103329, 159549, 143861,  37555,  27850,
         127396,  23958,  43466,  28576,  37552,  73926,  43681, 206601,  96398,
          25276,   8026,  34970, 211892, 119419, 145730, 121371, 100143, 198097,
         204802,  18949,  53375, 115478, 211400,   8023,  67427, 129516, 148628,
         172224,  18953,  54505,  41717, 169586, 205031,  25274,  82339, 170441,
           9446,  53718, 187162,  96397, 107750,  73932,  37545,  40389, 181471,
          68146, 195020,  13493,  43680,   8031,  94629,   8024,   8027, 172023,
          85728, 156000,  73930, 183859, 114789,  21252,  38388, 200754, 172376,
          84770]])

Попробуем предсказать всех

In [116]:
predictions = []
num_not_found = 0
zero_topk = 0

dataset_uids = dataset.field2token_id[dataset.uid_field]

for user, vacs in tqdm(zip(test_users, test_vacancies), total=len(test_users)):
#    print("Try: ", user)
#    if user not in dataset.field2token_id[dataset.uid_field]:   # либо в тесте есть такой юзер, которого не было в трейне, либо в трейне он отсеялся по порогу (?)
    if user not in dataset_uids:   # либо в тесте есть такой юзер, которого не было в трейне, либо в трейне он отсеялся по порогу (?)
        predictions.append(vacs)
        num_not_found += 1
#        print("Not found user: ", user)
        continue
    
#    cuser = dataset.token2id(dataset.uid_field, user)   # получили id очередного юзера по его имени

    _, topk_iid_list = predict_for_all_item(user, dataset, model)
#    _, topk_iid_list = full_sort_topk([cuser], model, test_data, k=N_PREDICTIONS, device='cpu')
    if len(topk_iid_list) == 0:
#        print("Zero topk for user ", user)
        predictions.append(vacs)
        zero_topk += 1
        continue
    
    last_topk_iid_list = topk_iid_list[-1]
    recommendations = dataset.id2token(dataset.iid_field, last_topk_iid_list.cpu()).tolist()

    predictions.append(recommendations)

print("Not found: ", num_not_found)
print("Zero topk: ", zero_topk)


  0%|          | 0/83189 [00:00<?, ?it/s]

Not found:  30906
Zero topk:  0


Предскажем в сочетании ALS и нашей модели

In [58]:
predictions = []
num_not_found = 0
num_not_found_als = 0
zero_topk = 0

dataset_uids = dataset.field2token_id[dataset.uid_field]

for user, vacs in tqdm(zip(test_users, test_vacancies), total=len(test_users)):
#    print("Try: ", user)
#    if user not in dataset.field2token_id[dataset.uid_field]:   # либо в тесте есть такой юзер, которого не было в трейне, либо в трейне он отсеялся по порогу (?)
    if user not in dataset_uids:   # либо в тесте есть такой юзер, которого не было в трейне, либо в трейне он отсеялся по порогу (?) - тогда попробуем ALS
        if user not in user2idx:
             predictions.append(vacs)  # если его нет и тут больше ничего не остается
             num_not_found_als += 1
             continue
        cuser = user2idx[user]
        recommendations = als_model.recommend(cuser, uv_mat[cuser], N=N_PREDICTIONS, filter_already_liked_items=False)[0]
        recommendations = [idx2vac[cv] for cv in recommendations]
        predictions.append(recommendations)        
         
        num_not_found += 1
#        print("Not found user: ", user)
        continue
    
#    cuser = dataset.token2id(dataset.uid_field, user)   # получили id очередного юзера по его имени

    _, topk_iid_list = predict_for_all_item(user, dataset, model)
#    _, topk_iid_list = full_sort_topk([cuser], model, test_data, k=N_PREDICTIONS, device='cpu')
    if len(topk_iid_list) == 0:
#        print("Zero topk for user ", user)
        if user not in user2idx:
             predictions.append(vacs)  # если его нет и тут больше ничего не остается
             num_not_found_als += 1
             continue
        cuser = user2idx[user]
        recommendations = als_model.recommend(cuser, uv_mat[cuser], N=N_PREDICTIONS, filter_already_liked_items=False)[0]
        recommendations = [idx2vac[cv] for cv in recommendations]
        predictions.append(recommendations)        

        zero_topk += 1
        continue
    
    last_topk_iid_list = topk_iid_list[-1]
    recommendations = dataset.id2token(dataset.iid_field, last_topk_iid_list.cpu()).tolist()

    predictions.append(recommendations)

print("Not found: ", num_not_found)
print("Not found by ALS: ", num_not_found_als)
print("Zero topk: ", zero_topk)


100%|██████████| 83189/83189 [51:15<00:00, 27.05it/s]  


Not found:  30906
Not found by ALS:  0
Zero topk:  0


In [ ]:
test = test.with_columns(pl.lit(pl.Series(predictions)).alias('predictions'))
test.select(['user_id', 'session_id', 'predictions']).write_parquet('sasrecf-als-combined_submission.pq')

In [ ]:
# test = test.with_columns(pl.Series(name='predictions', values=predictions))
# в чате группы предлагали так


In [132]:
dataset_uids = dataset.field2token_id[dataset.uid_field]

In [131]:
predictions = []
num_not_found = 0
zero_topk = 0

In [104]:
def final_predict(user_vacs_tuple):
    global num_not_found, zero_topk
    user, vacs = user_vacs_tuple
#    print("Try: ", user)
    if user not in dataset_uids:   # либо в тесте есть такой юзер, которого не было в трейне, либо в трейне он отсеялся по порогу (?)
        num_not_found += 1
#        print("Not found user: ", user)
        return vacs.copy()
#    print("User found ok: ", user)
    
#    cuser = dataset.token2id(dataset.uid_field, user)   # получили id очередного юзера по его имени

    _, topk_iid_list = predict_for_all_item(user, dataset, model)
#    print("Predictions got for user ", user)    
    if len(topk_iid_list) == 0:
#        print("Zero topk for user ", user)
        zero_topk += 1
        return vacs.copy()

#    print("Prediction not zero for user ", user)

    
    last_topk_iid_list = topk_iid_list[-1]
    recommendations = dataset.id2token(dataset.iid_field, last_topk_iid_list.cpu()).tolist()
#    print("Recommendations ok...")
   
    return recommendations

In [126]:
from multiprocessing import Pool

In [122]:
tu = test_users[:8]
tv = test_vacancies[:8]

In [127]:
print("Not found: ", num_not_found)
print("Zero topk: ", zero_topk)

Not found:  2
Zero topk:  0


In [130]:
pool = Pool(processes=8)
# predictions = list(tqdm(pool.imap(final_predict, zip(test_users, test_vacancies), chunksize=8), total=len(test_users)))
#predictions = list(tqdm(pool.imap(final_predict, zip(test_users, test_vacancies), chunksize=8), total=8))
predictions = list(pool.imap(final_predict, zip(tu, tv)))


KeyboardInterrupt: 

In [50]:
tu = test_users[:8]
tv = test_vacancies[:8]

In [58]:
tu

['u_482520',
 'u_582132',
 'u_212584',
 'u_425177',
 'u_700997',
 'u_687201',
 'u_573219',
 'u_1082782']

In [51]:
vac = tv[0]

In [65]:
vac2 = vac.copy()
vac2

['v_2597196', 'v_1223061']

In [103]:
# pool = Pool(processes=2)
# predictions = list(tqdm(pool.imap(final_predict, zip(test_users, test_vacancies), chunksize=8), total=len(test_users)))
#predictions = list(tqdm(pool.imap(final_predict, zip(tu, tv))))
# predictions = list(pool.map(final_predict, zip(tu, tv)))
# result = pool.map(final_predict, zip(tu, tv))
results_iterator = imap_easy(final_predict, zip(tu, tv), -1, 8)
#predictions = list(result)
predictions = list(results_iterator)


KeyboardInterrupt: 

In [121]:
test = test.with_columns(pl.lit(pl.Series(predictions)).alias('predictions'))
test.select(['user_id', 'session_id', 'predictions']).write_parquet('sasrecf-10-15_good-dataset_last_item_submission.pq')

In [ ]:
# config, model, dataset, train_data, valid_data, test_data

In [ ]:
dataset.token2id()

In [14]:
test

user_id,session_id,vacancy_id,action_type,action_dt
str,str,list[str],list[i64],list[datetime[ns]]
"""u_482520""","""s_25018731""","[""v_2597196"", ""v_1223061"", ""v_1223061""]","[2, 2, 1]","[2023-11-19 12:03:13.089, 2023-11-19 12:03:30.396, 2023-11-19 12:05:03.473]"
"""u_582132""","""s_481216""","[""v_470400"", ""v_470400"", ""v_1530783""]","[2, 1, 2]","[2023-11-21 15:39:47.981, 2023-11-21 15:43:57.620, 2023-11-21 15:44:37.982]"
"""u_212584""","""s_16918781""","[""v_1572055"", ""v_1572055"", … ""v_1329494""]","[2, 3, … 1]","[2023-11-16 08:41:47.031, 2023-11-16 08:45:21.403, … 2023-11-16 09:05:27.379]"
"""u_425177""","""s_17505104""","[""v_1375331"", ""v_1922852""]","[2, 2]","[2023-11-17 12:42:18.513, 2023-11-17 12:43:52.744]"
"""u_700997""","""s_15528830""","[""v_2152997"", ""v_2152997"", ""v_1217630""]","[2, 1, 2]","[2023-11-16 17:22:53.530, 2023-11-16 17:23:19.885, 2023-11-16 17:23:33.472]"
"""u_687201""","""s_11187326""","[""v_2602959"", ""v_549337"", … ""v_212060""]","[2, 2, … 1]","[2023-11-21 14:41:30.546, 2023-11-21 15:00:25.506, … 2023-11-21 15:24:56.271]"
"""u_573219""","""s_27394215""","[""v_1836184"", ""v_2733253"", … ""v_2297339""]","[1, 2, … 2]","[2023-11-21 18:48:43.424, 2023-11-21 18:49:12.552, … 2023-11-21 19:48:55.704]"
"""u_1082782""","""s_30341164""","[""v_1026036"", ""v_1026036"", … ""v_206700""]","[2, 1, … 2]","[2023-11-17 02:54:19.561, 2023-11-17 02:54:50.486, … 2023-11-17 03:21:22.008]"
"""u_81444""","""s_1909964""","[""v_2497384"", ""v_1480870"", … ""v_1500697""]","[2, 2, … 1]","[2023-11-15 12:38:34.579, 2023-11-15 13:03:41.360, … 2023-11-15 13:05:15.916]"


In [ ]:
external_user_ids = dataset.id2token(dataset.uid_field, list(range(dataset.user_num)))[1:]   #fist element in array is 'PAD'(default of Recbole) ->remove it 

In [ ]:
external_user_ids

In [ ]:
external_item_list

In [ ]:
train = pl.read_parquet("./hh_recsys_train_hh.pq")
test = pl.read_parquet("./hh_recsys_test_hh.pq")

In [ ]:
unique_users = train['user_id'].unique().to_list()
unique_vacancies = train['vacancy_id'].explode().unique().to_list()

user2idx = {user_id: idx for idx, user_id in enumerate(unique_users)}
vac2idx = {vac_id: idx for idx, vac_id in enumerate(unique_vacancies)}

idx2vac = {idx: vac_id for vac_id, idx in vac2idx.items()}
idx2user = {idx: user_id for user_id, idx in user2idx.items()}


In [ ]:
test_users = test['user_id'].to_list()
test_vacancies = test.select(pl.col('vacancy_id').list.unique(maintain_order=True))['vacancy_id'].to_list()

In [ ]:
predictions = []

for user, vacs in tqdm(zip(test_users, test_vacancies), total=len(test_users)):
    if user not in user2idx:
        predictions.append(vacs)
        continue
    cuser = user2idx[user]

    recommendations = als_model.recommend(cuser, uv_mat[cuser], N=N_PREDICTIONS)[0]
    recommendations = [idx2vac[cv] for cv in recommendations]
    predictions.append(recommendations)

In [ ]:
#test = test.with_columns(pl.lit(predictions).alias('predictions'))
test = test.with_columns(pl.lit(pl.Series(predictions)).alias('predictions'))
test.select(['user_id', 'session_id', 'predictions']).write_parquet('als_submission.pq')

In [118]:
test

{'vacancy_id_list': tensor([[ 84548,  84548, 104162, 104162, 201448,  25052,  24355,  25052,  25052,
           47335]]),
 'item_length': tensor([10])}

In [1]:
from tqdm import tqdm
from multiprocessing import Pool

In [2]:
test_users2 = [1, 2, 3]
test_vacancies2 = [[11,111,1111], [22,222,2222], [33,333,3333]]

In [3]:
def func2(param1_tuple):
  param1, param2 = param1_tuple
  print("param1 =", param1, "param2 = ", param2)
  return param2

In [7]:
pool = Pool(processes=8)

In [8]:
mapped_values = list(tqdm(pool.map(func2, zip(test_users2, test_vacancies2), chunksize=2), total=len(test_users2)))

param1 =param1 =  31  param2 = param2 =   [11, 111, 1111][33, 333, 3333]

param1 = 2 param2 =  [22, 222, 2222]


100%|██████████| 3/3 [00:00<00:00, 33376.42it/s]


In [6]:
mapped_values

[[11, 111, 1111], [22, 222, 2222], [33, 333, 3333]]

In [7]:
with open('dataset_test.pkl', 'wb') as fp:
    pickle.dump(dataset, fp)

Уберем из предсказаний те, на которые он уже откликался

In [6]:
import pickle

In [71]:
with open('predictions.pkl', 'rb') as fp:
    predictions = pickle.load(fp)

In [10]:
test = pl.read_parquet("./hh_recsys_test_hh.pq", low_memory=True)

In [30]:
test_users = test['user_id'].to_list()
test_vacancies_all = test.select(pl.col('vacancy_id'))['vacancy_id'].to_list()
test_actions_all = test.select(pl.col('action_type'))['action_type'].to_list()

In [59]:
test[0] 

user_id,session_id,vacancy_id,action_type,action_dt
str,str,list[str],list[i64],list[datetime[ns]]
"""u_482520""","""s_25018731""","[""v_2597196"", ""v_1223061"", ""v_1223061""]","[2, 2, 1]","[2023-11-19 12:03:13.089, 2023-11-19 12:03:30.396, 2023-11-19 12:05:03.473]"


In [23]:
test[0]['vacancy_id'].to_list()[0]

['v_2597196', 'v_1223061', 'v_1223061']

In [66]:
len(predictions[1][0])

99

In [72]:
new_predictions = []

for vacs, acts, prediction in zip(test_vacancies_all, test_actions_all, predictions):
    if 1 in acts:
        indxs = [index for index, value in enumerate(acts) if value == 1]  # индексы откликов в action_type - надо убрать рекомендации этих вакансий из prediction
        for i in sorted(indxs, reverse=True):
            del prediction[0][i]
            del prediction[1][i]
        
    new_predictions.append(prediction[1])


In [76]:
!pwd

/media/arseny/Диск D/Обучение Data Science/Хакатон RecSys


In [73]:
test = test.with_columns(pl.lit(pl.Series(new_predictions)).alias('predictions'))
test.select(['user_id', 'session_id', 'predictions']).write_parquet('without_answered_vacs_submission.pq')

In [74]:
test

user_id,session_id,vacancy_id,action_type,action_dt,predictions
str,str,list[str],list[i64],list[datetime[ns]],list[str]
"""u_482520""","""s_25018731""","[""v_2597196"", ""v_1223061"", ""v_1223061""]","[2, 2, 1]","[2023-11-19 12:03:13.089, 2023-11-19 12:03:30.396, 2023-11-19 12:05:03.473]","[""v_1943657"", ""v_802227"", … ""v_1209207""]"
"""u_582132""","""s_481216""","[""v_470400"", ""v_470400"", ""v_1530783""]","[2, 1, 2]","[2023-11-21 15:39:47.981, 2023-11-21 15:43:57.620, 2023-11-21 15:44:37.982]","[""v_1530783"", ""v_1919088"", … ""v_483786""]"
"""u_212584""","""s_16918781""","[""v_1572055"", ""v_1572055"", … ""v_1329494""]","[2, 3, … 1]","[2023-11-16 08:41:47.031, 2023-11-16 08:45:21.403, … 2023-11-16 09:05:27.379]","[""v_1329494"", ""v_1592343"", … ""v_1540776""]"
"""u_425177""","""s_17505104""","[""v_1375331"", ""v_1922852""]","[2, 2]","[2023-11-17 12:42:18.513, 2023-11-17 12:43:52.744]","[""v_1922852"", ""v_350120"", … ""v_2045008""]"
"""u_700997""","""s_15528830""","[""v_2152997"", ""v_2152997"", ""v_1217630""]","[2, 1, 2]","[2023-11-16 17:22:53.530, 2023-11-16 17:23:19.885, 2023-11-16 17:23:33.472]","[""v_897402"", ""v_1796672"", … ""v_530167""]"
"""u_687201""","""s_11187326""","[""v_2602959"", ""v_549337"", … ""v_212060""]","[2, 2, … 1]","[2023-11-21 14:41:30.546, 2023-11-21 15:00:25.506, … 2023-11-21 15:24:56.271]","[""v_212060"", ""v_1878247"", … ""v_2428925""]"
"""u_573219""","""s_27394215""","[""v_1836184"", ""v_2733253"", … ""v_2297339""]","[1, 2, … 2]","[2023-11-21 18:48:43.424, 2023-11-21 18:49:12.552, … 2023-11-21 19:48:55.704]","[""v_450284"", ""v_1199926"", … ""v_123110""]"
"""u_1082782""","""s_30341164""","[""v_1026036"", ""v_1026036"", … ""v_206700""]","[2, 1, … 2]","[2023-11-17 02:54:19.561, 2023-11-17 02:54:50.486, … 2023-11-17 03:21:22.008]","[""v_2373713"", ""v_1507296"", … ""v_478722""]"
"""u_81444""","""s_1909964""","[""v_2497384"", ""v_1480870"", … ""v_1500697""]","[2, 2, … 1]","[2023-11-15 12:38:34.579, 2023-11-15 13:03:41.360, … 2023-11-15 13:05:15.916]","[""v_1498100"", ""v_344065"", … ""v_1418492""]"


In [75]:
test.select(['user_id', 'session_id', 'predictions']).write_parquet('without_answered_vacs_submission.pq')

## Предсказания только на основе истории, для неизвестных пользователей/без пользователей

In [4]:
!ls saved

ALS-200-fulldata.npz		  SASRec-Jan-04-2024_07-58-21.pth
ALS-200-fulldata.pkl		  SASRec-Jan-06-2024_02-07-14.pth
FPMC-Jan-14-2024_15-27-07.pth	  SASRec-Jan-16-2024_01-44-14.pth
ItemKNN-Jan-23-2024_15-30-10.pth  SHAN-Jan-10-2024_12-13-04.pth
ItemKNN-Jan-26-2024_15-20-50.pth  SINE-Jan-23-2024_18-05-07.pth
SASRecF-Jan-27-2024_15-13-17.pth


In [4]:
import cloudpickle
from recbole.data import create_dataset
import torch

In [11]:
model_file_path = "saved/SASRec-Jan-16-2024_01-44-14.pth"

In [14]:
dataset

hh_recsys
The number of users: 291499
Average actions of users: 38.2871237538508
The number of items: 214819
Average actions of items: 51.953839994786286
The number of inters: 11160620
The sparsity of the dataset: 99.98217709215284%
Remain Fields: ['user_id', 'vacancy_id', 'action_type', 'timestamp', 'name', 'company_id', 'keySkills', 'compensation_from', 'compensation_to', 'area_id', 'area_regionId', 'employment', 'workSchedule', 'workExperience']

In [11]:
checkpoint['state_dict']['item_embedding.weight'].shape

torch.Size([271213, 64])

In [12]:
dataset

hh_recsys
The number of users: 291499
Average actions of users: 38.2871237538508
The number of items: 214819
Average actions of items: 51.953839994786286
The number of inters: 11160620
The sparsity of the dataset: 99.98217709215284%
Remain Fields: ['user_id', 'vacancy_id', 'action_type', 'timestamp', 'name', 'company_id', 'keySkills', 'compensation_from', 'compensation_to', 'area_id', 'area_regionId', 'employment', 'workSchedule', 'workExperience']

In [3]:
from typing import List, Tuple

import numpy as np
import torch
from fastapi.applications import FastAPI
from pydantic import BaseModel
from recbole.data import create_dataset
from recbole.data.dataset.sequential_dataset import SequentialDataset
from recbole.data.interaction import Interaction
from recbole.model.sequential_recommender.sine import SINE
from recbole.utils import get_model, init_seed

In [18]:
def load_model0(model_file: str, dataset_file: str) -> Tuple[SINE, SequentialDataset]:   # не работает?
    with open(dataset_file, "rb") as f:
        dataset = cloudpickle.load(f)

    checkpoint = torch.load(model_file)
    config = checkpoint["config"]
    init_seed(config["seed"], config["reproducibility"])
    model = get_model(config["model"])(config, dataset).to(config["device"])
    model.load_state_dict(checkpoint["state_dict"])
    model.load_other_parameter(checkpoint.get("other_parameter"))
    return model, dataset


In [12]:
def load_model(model_file: str) -> Tuple[SINE, SequentialDataset]:
    checkpoint = torch.load(model_file)
    config = checkpoint["config"]
    init_seed(config["seed"], config["reproducibility"])
    dataset = create_dataset(config) # при размере модели 199М dataset получился 488М и работало такое сохранение 6 минут
    model = get_model(config["model"])(config, dataset).to(config["device"])
    model.load_state_dict(checkpoint["state_dict"])
    model.load_other_parameter(checkpoint.get("other_parameter"))
    return model, dataset


In [14]:
model_sine, dataset_sine = load_model(
    model_file="saved/SASRec-Jan-06-2024_02-07-14.pth"
    )

RuntimeError: Error(s) in loading state_dict for SASRec:
	size mismatch for item_embedding.weight: copying a param with shape torch.Size([435832, 32]) from checkpoint, the shape in current model is torch.Size([382539, 32]).

The number of items: 214819 если item_inter_num_interval = [15,inf)

In [ ]:
class ItemHistory(BaseModel):
    sequence: List[str]
    topk: int

In [ ]:
class RecommendedItems(BaseModel):
    score_list: List[float]
    item_list: List[str]


In [ ]:
def sine_user_to_item(item_history: ItemHistory):
    item_history_dict = item_history.dict()
    item_sequence = item_history_dict["sequence"]
    item_length = len(item_sequence)
    pad_length = 50  # pre-defined by recbole

    padded_item_sequence = torch.nn.functional.pad(
        torch.tensor(dataset.token2id(dataset.iid_field, item_sequence)),
        (0, pad_length - item_length),
        "constant",
        0,
    )

    input_interaction = Interaction(
        {
            "item_list": padded_item_sequence.reshape(1, -1),
            "item_length": torch.tensor([item_length]),
        }
    )
    scores = model.full_sort_predict(input_interaction.to(model.device))
    scores = scores.view(-1, dataset.item_num)
    scores[:, 0] = -np.inf  # pad item score -> -inf
    topk_score, topk_iid_list = torch.topk(scores, item_history_dict["topk"])

    predicted_score_list = topk_score.tolist()[0]
    predicted_item_list = dataset.id2token(
        dataset.iid_field, topk_iid_list.tolist()
    ).tolist()

    recommended_items = {
        "score_list": predicted_score_list,
        "item_list": predicted_item_list,
    }
    return recommended_items

## Попробуем объединить предсказания разных моделей

In [4]:
with open('predictions-sine.pkl', 'rb') as f:
    predictions_sine = pickle.load(f)


In [5]:
with open('predictions-shan.pkl', 'rb') as f:
    predictions_shan = pickle.load(f)


In [6]:
with open('predictions-sasrecf.pkl', 'rb') as f:
    predictions_sasrecf = pickle.load(f)


In [7]:
with open('predictions-fdsa.pkl', 'rb') as f:
    predictions_fdsa = pickle.load(f)


In [ ]:
predictions_sine[1]

In [137]:
full_prediction = zip(predictions_sasrecf, predictions_shan, predictions_sine, predictions_fdsa)
#full_prediction = zip(predictions_sasrecf, predictions_shan, predictions_sine, predictions_fdsa)

In [ ]:
# Здесь выбираем предсказания с наилучшими скорами. Но работает плохо.
final_predictions = []

for itm in tqdm(full_prediction, total=len(predictions_sasrecf)):
    scores_i = []
    vacancies_i = []
    for i,pred in enumerate(itm):
        scores_i += itm[i][0]
        vacancies_i += itm[i][1]
#    vacs_sorted = [x[1] for x in sorted(zip(scores_i,vacancies_i), key=lambda tup: tup[0], reverse=True) ]
    vacs_sorted = []
    [vacs_sorted.append(x[1]) for x in sorted(zip(scores_i,vacancies_i), key=lambda tup: tup[0], reverse=True) if x[1] not in vacs_sorted ]
    vacs_sorted=vacs_sorted[:N_PREDICTIONS]

    final_predictions.append(vacs_sorted)




In [10]:
# Здесь выбираем сумму предсказаний по разным моделям.
final_predictions = []
lengths = [] # замерим длины суммарных списков, т.е. насколько разнятся предсказания

for itm in tqdm(full_prediction, total=len(predictions_sasrecf)):  # в itm у нас предсказания от n разных моделей для конкретного юзера
    scores_i = {}
    vacancies_i = []
    for _,pred in enumerate(itm):  # pred будет tuple из 2 элементов-списков: скоры и вакансии
        for sc,vac in zip(pred[0], pred[1]):
            scores_i[vac] = scores_i.get(vac, 0.0) + sc
#        print(scores_i)
        
    vacs_sorted = [x[0] for x in sorted(scores_i.items(), key=lambda tup: tup[1], reverse=True) ]
    
    lengths.append(len(vacs_sorted))
    
    vacs_sorted=vacs_sorted[:N_PREDICTIONS]
    
    final_predictions.append(vacs_sorted)
    





100%|██████████| 83189/83189 [00:10<00:00, 7746.33it/s]


In [36]:
# Здесь выбираем сумму предсказаний по разным моделям.
final_predictions = []
lengths = [] # замерим длины суммарных списков, т.е. насколько разнятся предсказания

for itm in tqdm(full_prediction, total=len(predictions_sasrecf)):  # в itm у нас предсказания от n разных моделей для конкретного юзера
    scores_i = {}
    vacancies_i = []
    for _,pred in enumerate(itm):  # pred будет tuple из 2 элементов-списков: скоры и вакансии
        min_score = pred[0][-1]
        for sc,vac in zip(pred[0], pred[1]):
            scores_i[vac] = scores_i.get(vac, min_score) + sc
        
    vacs_sorted = [x[0] for x in sorted(scores_i.items(), key=lambda tup: tup[1], reverse=True) ]
    
    lengths.append(len(vacs_sorted))
    
    vacs_sorted=vacs_sorted[:N_PREDICTIONS]
    
    final_predictions.append(vacs_sorted)


  1%|          | 793/83189 [00:00<00:10, 7929.16it/s]

100%|██████████| 83189/83189 [00:11<00:00, 7034.89it/s]


In [130]:
full_prediction = zip(predictions_sasrecf, predictions_shan)

In [138]:
# Здесь выбираем сумму предсказаний по разным моделям и проверяем, смотрел ли он такую вакансию.
final_predictions = []
lengths = [] # замерим длины суммарных списков, т.е. насколько разнятся предсказания

for i, itm in tqdm(enumerate(full_prediction), total=len(predictions_sasrecf)):  # в itm у нас предсказания от n разных моделей для конкретного юзера
    scores_i = {}
    vacancies_i = []
    activity = test[i]
    activity_vacs = activity['vacancy_id'][0].to_numpy()
    activity_interactions = activity['action_type'][0].to_numpy()
    for _,pred in enumerate(itm):  # pred будет tuple из 2 элементов-списков: скоры и вакансии
        min_score = pred[0][-1]
        for sc,vac in zip(pred[0], pred[1]):   # sc - скор каждой вакансии-кандидата, vac - сама эта вакансия
            if vac not in activity_vacs : # юзер с такой вакансией вообще не взаимодействовал
                sc = sc / 4.0  # было 3
            else:  # надо посмотреть, как взаимодействовал, если "1" то вычеркнуть или снизить скор
                indxs = np.where(activity_vacs == vac)[0]
                if 1 in activity_interactions[indxs]:
                    sc = sc / 10.0
                elif 3 in activity_interactions[indxs]:  # 3 - положил в избранные
                    sc = sc * 2.0

            scores_i[vac] = scores_i.get(vac, min_score) + sc
        
    vacs_sorted = [x[0] for x in sorted(scores_i.items(), key=lambda tup: tup[1], reverse=True) ]
    
    lengths.append(len(vacs_sorted))
    
    vacs_sorted=vacs_sorted[:N_PREDICTIONS]
    
    final_predictions.append(vacs_sorted)


100%|██████████| 83189/83189 [02:48<00:00, 493.85it/s]


In [139]:
len(itm)

4

In [140]:
test = pl.read_parquet("./hh_recsys_test_hh.pq")

In [141]:
test = test.with_columns(pl.lit(pl.Series(final_predictions)).alias('predictions'))
test.select(['user_id', 'session_id', 'predictions']).write_parquet('summary2_submission.pq')

In [142]:
test

user_id,session_id,vacancy_id,action_type,action_dt,predictions
str,str,list[str],list[i64],list[datetime[ns]],list[str]
"""u_482520""","""s_25018731""","[""v_2597196"", ""v_1223061"", ""v_1223061""]","[2, 2, 1]","[2023-11-19 12:03:13.089, 2023-11-19 12:03:30.396, 2023-11-19 12:05:03.473]","[""v_2597196"", ""v_802227"", … ""v_1372471""]"
"""u_582132""","""s_481216""","[""v_470400"", ""v_470400"", ""v_1530783""]","[2, 1, 2]","[2023-11-21 15:39:47.981, 2023-11-21 15:43:57.620, 2023-11-21 15:44:37.982]","[""v_1530783"", ""v_1347362"", … ""v_751423""]"
"""u_212584""","""s_16918781""","[""v_1572055"", ""v_1572055"", … ""v_1329494""]","[2, 3, … 1]","[2023-11-16 08:41:47.031, 2023-11-16 08:45:21.403, … 2023-11-16 09:05:27.379]","[""v_1572055"", ""v_1846862"", … ""v_575528""]"
"""u_425177""","""s_17505104""","[""v_1375331"", ""v_1922852""]","[2, 2]","[2023-11-17 12:42:18.513, 2023-11-17 12:43:52.744]","[""v_1922852"", ""v_459362"", … ""v_2087770""]"
"""u_700997""","""s_15528830""","[""v_2152997"", ""v_2152997"", ""v_1217630""]","[2, 1, 2]","[2023-11-16 17:22:53.530, 2023-11-16 17:23:19.885, 2023-11-16 17:23:33.472]","[""v_1217630"", ""v_466981"", … ""v_192191""]"
"""u_687201""","""s_11187326""","[""v_2602959"", ""v_549337"", … ""v_212060""]","[2, 2, … 1]","[2023-11-21 14:41:30.546, 2023-11-21 15:00:25.506, … 2023-11-21 15:24:56.271]","[""v_2322652"", ""v_1648353"", … ""v_2450990""]"
"""u_573219""","""s_27394215""","[""v_1836184"", ""v_2733253"", … ""v_2297339""]","[1, 2, … 2]","[2023-11-21 18:48:43.424, 2023-11-21 18:49:12.552, … 2023-11-21 19:48:55.704]","[""v_1828011"", ""v_1281516"", … ""v_1574143""]"
"""u_1082782""","""s_30341164""","[""v_1026036"", ""v_1026036"", … ""v_206700""]","[2, 1, … 2]","[2023-11-17 02:54:19.561, 2023-11-17 02:54:50.486, … 2023-11-17 03:21:22.008]","[""v_206700"", ""v_2279678"", … ""v_2241366""]"
"""u_81444""","""s_1909964""","[""v_2497384"", ""v_1480870"", … ""v_1500697""]","[2, 2, … 1]","[2023-11-15 12:38:34.579, 2023-11-15 13:03:41.360, … 2023-11-15 13:05:15.916]","[""v_1478745"", ""v_1312299"", … ""v_1565146""]"


In [11]:
min(lengths)

128

In [12]:
max(lengths)

400

In [13]:
import statistics
statistics.mean(lengths) 

254.25648823762756

In [14]:
statistics.median(lengths)

249